In [ ]:
# @title Setup
from google.cloud import bigquery
from google.colab import data_table
import bigframes.pandas as bpd
import pandas as pd
import datetime
import requests
import os

project = 'livil-flags-messaging-hub' # Project ID inserted based on the query results selected to explore
location = 'EU' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()

In [ ]:
from google.cloud import bigquery, storage

# Step 1: Load SQL query from GCS (.sql file)
def load_query_from_gcs(bucket_name, blob_name):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)
    query = blob.download_as_text()
    return query

# Your GCS location of the .sql file
bucket_name = 'ico_new_users_report'
blob_name = 'new_users_query/New_Users_V05.sql'

# Step 2: Read the query
sql_query = load_query_from_gcs(bucket_name, blob_name)

# Step 3: Run the query using BigQuery client
client = bigquery.Client()
job = client.query(sql_query)
job.result()  # Wait for the query to finish

# Step 4: Get the destination table (automatically created temp table)
destination = job.destination

# Step 5: Load data from a BigQuery table using BigFrames DataFrame
bq_df = bpd.read_gbq(f"{destination.project}.{destination.dataset_id}.{destination.table_id}")


bq_df = bq_df.sort_values(by='First_Open', ascending=False)

# Step 6: Use the dataframe
#bq_df  # Displays the BigFrames DataFrame


In [ ]:
# Convert BigQuery DataFrame to pandas DataFrame.
pandas_df = bq_df.to_pandas()

In [ ]:
now = datetime.datetime.now()
timestamp_str = now.strftime("%Y-%m-%d_%H-%M")
today_date = now.strftime("%Y-%m-%d")
file_name = f"New_Users_Report_{timestamp_str}.xlsx"
pandas_df.to_excel(file_name, index=False)

In [ ]:
!pip install slack-sdk==3.35.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.3/293.3 kB 29.2 MB/s eta 0:00:00


In [ ]:
# Slack details
slack_token = ''
channel_id = ''

In [ ]:
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError

In [ ]:
def upload_file_to_slack(token, channel_id, file_path, title, today_date):

    client = WebClient(token=token)

    try:
        response = client.files_upload_v2(
            channel=channel_id,
            file=file_path,
            title=title,
            initial_comment=f"*From scheduler New Users Report* 📊"
        )
        permalink = response["file"]["permalink"]
        print(f"✅ File uploaded successfully: {permalink}")
        return permalink

    except SlackApiError as e:
        print(f"❌ Error uploading file: {e.response['error']}" )
        return None

In [ ]:
upload_file_to_slack(slack_token, channel_id, file_name, file_name, today_date)

✅ File uploaded successfully: https://livilco.slack.com/files/U08P9JDG31U/F09MG6VEFRQ/new_users_report_2025-10-20_12-19.xlsx


'https://livilco.slack.com/files/U08P9JDG31U/F09MG6VEFRQ/new_users_report_2025-10-20_12-19.xlsx'